In [1]:
from labyak import LabJack
labjack = LabJack(devid='470017907')

Connected to LabJack (470017907).


# Analog I/O
Here's a simple loopback test where we generate 1 V on channel DAC0 and measure it on ADC0:

In [2]:
labjack.analog.AOut(0, 1)
labjack.analog.AIn(0)

1.0014537572860718

The LJTick-DAC (TDAC) adapter board can generate voltages from -10 to 10 V. Here's another loopback test where we generate -6 V on TDAC0 and measure it on ADC1:

In [3]:
labjack.analog.TDAC(0, -6)
labjack.analog.AIn(1)

-6.002580165863037

# Digital I/O
Here's a loopback test where we output a series of bits on FIO2 and read them on FIO3:

In [5]:
write_bits = [0, 1, 1, 0, 1]
read_bits = []

for bit in write_bits:
    labjack.digital.DOut(2, bit)
    read_bits.append(labjack.digital.DIn(3))
    
print(read_bits)

[1, 1, 1, 1, 1]


# Temperature sensing
The LabJack can be used with type J or K thermocouples to measure temperatures. Here's an example using channels AIN2 and AIN3 for the positive and negative leads of a type J thermocouple:

In [ ]:
labjack.temperature.configure(pos_ch=2, neg_ch=3, kind='J')
labjack.temperature.TIn(2)

# Waveform generation
Now we'll demonstrate generation of arbitrary waveforms with the LabJack. Connect DAC1 to an oscilloscope and run the following:

In [6]:
import numpy as np
f = 5e3                                       # frequency of waveform
t = np.linspace(0, 1/f, 3000)                 # time axis of waveform
# V = 2.5*(1+np.sin(2*np.pi*f*t))               # sine wave
# V = 5*( t > t.max()/2 )                       # square wave
V = 5 * (t / t.max())                         # triangle wave

labjack.waveform.start(t, V, channels = [1])  # start generation

# Pattern generation
We can also use the LabJack as a digital pattern generator. Let's create a timing sequence where FIO3 goes high for 1 ms, low for 500 us, high for 2 ms, then low for 1 ms:

In [7]:
period=4.5e-3
sequence = {3: [(0, 1),         # start high
                (1e-3, 0),    
                (1.5e-3, 1),
                (3.5e-3, 0)]}

labjack.pattern.start(sequence, period)